In [184]:
#!/usr/bin/env python3
import os
import sys
import re
from Bio import AlignIO

af = "../mcm_test/pdb_bank/aligned/alignment_files/align_output.fasta" #sys.argv[1]
args=["902:1101","1735:1956","2608:2703","3495:3633","4189:4289","4891:5073"]


#translate numbers
i_start_list=list()
i_stop_list=list()
align = AlignIO.read(af, "fasta") 
pdbid="3JA8_stndrd" #os.environ["id"]
pdb=pdbid + ".pdb"
sequence=""
res_tick=0
tick=0
psuedo_n=0
translated=list()


for record in align:
    if record.id == pdbid:
        #get i set from residue_range
        for i in args: #sys.argv[2:]
            i_split=re.split(':', i)
            i_start=int(i_split[0])-1 #-1 because splice # takes places after character #
            i_stop=int(i_split[1])
            i_start_list.append(i_start+1)
            i_stop_list.append(i_stop)
            #spliced = record.seq[i_start:i_stop]
            #sequence=sequence+spliced
        #translate alignment column # to pdb res #
        for n in range(1, i_stop_list[-1]+1):
            subtractor=0 #the amount of gaps in the last encountered stretch of gaps
            #count residues 
            if record.seq[n-1:n] == '-':
                if n == i_start_list[tick]:
                    res_tick=res_tick+1
                else:
                    pass
            else:
                res_tick=res_tick+1
            #put important residues in pdb
            if n == i_start_list[tick]:  
                for x in range(res_tick, (res_tick+
                                (1+i_stop_list[tick]-i_start_list[tick]))):
                    if psuedo_n < n:
                        psuedo_n=n
                    if record.seq[psuedo_n-1:psuedo_n] == '-':
                        subtractor=subtractor+1
                        translated.append("GAP")
                        #pass
                    else:
                        if subtractor==0:
                            res=x
                        else:
                            res=x-subtractor
                        translated.append(res)
                    psuedo_n=psuedo_n+1
                tick=tick+1
                if tick+1 > len(i_start_list):
                    break
#write new pdb         
if not translated:
    print(pdbid," does not contain any residues in the specified range. Omitting from rest of process")
    os.remove(pdb)
    sys.exit()
newpdb=pdbid + "-cut" + ".pdb"
f = open(newpdb, "w")
pdbfile = open(pdb, 'r')
prev_t=translated[0]
#for x in range(len(translated)): #find first non-GAP t
#    if prev_t == 'GAP':
#        prev_t=translated[x]
#    if prev_t != 'GAP':
#        break
ter=False
prev_t_index=0
terline='TER'+' '*77
gapchain=1
gapres=-1
prev_chain=None
#gapline='ATOM      0  X   GAP 0   0       nan     nan     nan    nan   nan'
def gapline_func(gapchain,gapres):
    gapline='ATOM      0  X   GAP '+str(gapchain)+str(gapres).rjust(4)+'       nan     nan     nan    nan   nan'
    return gapline
gapped=False
for line in pdbfile:
    if line.startswith('ATOM'):
        for t in translated:
            if int(line[22:26]) == t:
                if prev_chain == None: #name chain for getchain 
                    gapchain=line[21]
                elif prev_chain != line[21]:
                    gapchain=prev_chain
                elif prev_chain == line[21]:
                    gapchain=line[21]
                if prev_t != t:
                    if prev_t != 'GAP':
                        prev_t_index=prev_t_index+1
                    gapped=False
                    for x in range(len(translated)):
                        if 'GAP' == translated[prev_t_index]:
                            if not ter:    #prevents double TER
                                f.write(terline+'\n')
                                ter=True
                            if gapres==-1: #alternate gapres so each gap comes out as a different res when renaming res's
                                gapres=-2
                            elif gapres==-2:
                                gapres=-1
                            f.write(gapline_func(gapchain,gapres)+'\n')
                            prev_t_index=prev_t_index+1
                            gapped=True
                    if gapped: #write ter at the end of gap streak
                            f.write(terline+'\n')
                            ter=True
                    if prev_t != 'GAP':
                        if t != prev_t+1:
                                if not ter:    #prevents double TER
                                    f.write(terline+'\n')
                                    ter=True
                                    realter=False
                prev_t=t
                prev_chain=line[21]
                f.write(line)
                ter=False
                realter=False
            else:
                continue
    elif line.startswith('TER'):
        if not ter:
            f.write(terline+'\n')
            realter=True
            ter=True
    else:
        print("potentially strange line:",line)
        f.write('\n')

#write terminal gap streak
if prev_t != 'GAP':
    prev_t_index=prev_t_index+1
for x in range(len(translated)):
    try:
        if 'GAP' == translated[prev_t_index]:
            if not ter:    #prevents double TER
                f.write(terline+'\n')
                ter=True
            if gapres==-1: #alternate gapres so each gap comes out as a different res when renaming res's
                gapres=-2
            elif gapres==-2:
                gapres=-1
            f.write(gapline_func(gapchain,gapres)+'\n')
            prev_t_index=prev_t_index+1
            gapped=True
    except:
        break
if gapped: #write ter at the end of gap streak
        f.write(terline+'\n')
        ter=True
        
f.close()    


In [216]:
#!/usr/bin/env python3
import os
import sys
import re
from Bio import AlignIO

af = "../mcm_test/pdb_bank/aligned/alignment_files/align_output.fasta" #sys.argv[1]
align = AlignIO.read(af, "fasta") 
pdbid="3JA8_stndrd" #os.environ["id"]
pdb=pdbid + ".pdb"
prev_length="oogabooga"

atomlist = ["N","CA","C","O"] #sys.argv[2:]

#get alignment length and check to make sure its the same for each row
for record in align:
    if prev_length != "oogabooga":
        if len(record.seq) != prev_length:
            emsg = 'ERROR: rows in alignment output file differ in length\n'
            sys.stderr.write(emsg)
            sys.exit(1)
    if record.id == pdbid:
        alignment_length=len(record.seq)
    prev_length=len(record.seq)
    
if alignment_length > 9999:
            emsg = 'ERROR: alignment length is greater than 9999, so it cannot fit in 1 pdb\n'
            sys.stderr.write(emsg)
            sys.exit(1)

#write new pdb         
newpdb=pdbid + "_withgaps" + ".pdb"
f = open(newpdb, "w")
pdbfh = open(pdb, 'r')
ter=False
terline='TER'+' '*77
gapchain=1
gapres=-1
gapatom='X'
prev_chain=None
col_index=1
prev_res=99999999
#gapline='ATOM      0  X   GAP 0   0       nan     nan     nan    nan   nan'
def gapline_func(gapchain,gapres,gapatom):
    gapline='ATOM      0  '+str(gapatom).ljust(3)+' GAP '+str(gapchain)+str(gapres).rjust(4)+'       nan     nan     nan    nan   nan'
    return gapline
gapped=False
for record in align:
    if record.id == pdbid:
        for line in pdbfh:
            if line.startswith('ATOM'):
                if line[12:16].strip() not in atomlist:
                    continue
                if prev_res==int(line[22:26]): #if same res, write line and avoid iterating through alignment columns
                    f.write(line)
                    ter=False
                    gapped=False
                    continue
                for n in range(col_index, alignment_length+1):
                    if record.seq[n-1:n] == '-':
                        #write gapline
                        if prev_chain == None: #name chain for getchain 
                            gapchain=line[21]
                        elif prev_chain != line[21]:
                            gapchain=prev_chain
                        elif prev_chain == line[21]:
                            gapchain=line[21]
                        if not ter:    #write TER at start of gap streak if there isnt one already
                            f.write(terline+'\n')
                            ter=True
                        if gapres==-1: #alternate gapres so each gap comes out as a different res when renaming res's
                            gapres=-2
                        elif gapres==-2:
                            gapres=-1
                        for gapatom in atomlist: 
                            f.write(gapline_func(gapchain,gapres,gapatom)+'\n')
                        col_index=col_index+1
                        gapped=True
                    else:
                        if gapped: #write ter at the end of gap streak
                            f.write(terline+'\n')
                            ter=True
                        #write line
                        prev_chain=line[21]
                        f.write(line)
                        prev_res=int(line[22:26])
                        gapped=False
                        col_index=col_index+1
                        ter=False
                        break
            elif line.startswith('TER'):
                if not ter:
                    f.write(terline+'\n')
                    ter=True
            else:
                print("potentially strange line:",line)
                f.write('\n')
                continue
        #write term gaps
        gapchain=prev_chain
        for n in range(col_index, alignment_length+1):
            if record.seq[n-1:n] == '-':
                if not ter:    #write TER at start of gap streak if there isnt one already
                    f.write(terline+'\n')
                    ter=True
                if gapres==-1: #alternate gapres so each gap comes out as a different res when renaming res's
                    gapres=-2
                elif gapres==-2:
                    gapres=-1
                for gapatom in atomlist: 
                    f.write(gapline_func(gapchain,gapres,gapatom)+'\n')
                col_index=col_index+1
                gapped=True
            else:
                emsg = 'ERROR: there are non-gap columns after the last line in pdb {} \n'.format(pdbid)
                sys.stderr.write(emsg)
                sys.exit(1)

        if gapped: #write ter at the end of gap streak
            if not ter:
                f.write(terline+'\n')
                ter=True
    else:
        continue


        
f.close()    
pdbfh.close()

In [220]:
#print(translated)
#print(prev_t_index)

In [170]:
print(translated[prev_t_index])

GAP


In [186]:
len(translated)

941

In [142]:
#904-1100
print(res_tick, (res_tick+(1+i_stop_list[0]-i_start_list[0]))) #(1+i_stop_list[0]-i_start_list[0]) is the amount of columns in the i_range

print(i_stop_list[0],i_start_list[0])

268 465
1100 904


In [146]:
print(res_tick, (res_tick+(1+i_stop_list[0]-i_start_list[0])))
print(i_stop_list[0],i_start_list[0])

268 467
1100 902


In [196]:
af = "../mcm_test/pdb_bank/aligned/alignment_files/align_output.fasta"
align = AlignIO.read(af, "fasta") 
#alignment_length=len
for record in align:
    print(len(record.seq))
    if record.id == pdbid:
        print(len(record.seq))

5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271
5271


In [219]:
#!/usr/bin/env python3
import os
import sys
#sys.argv[1]
pdbid = "../mcm_test/pdb_bank/cut_and_cleaned/withoutgaps/6U0M_ltr_stndrd" #os.environ["id"]
pdbf = pdbid+".pdb"
pdbfh = open(pdbf, 'r')
prev_chain = "oogabooga"
chainlist=list()
for line in pdbfh:
    if line.startswith('ATOM'):
        if line[21] != prev_chain:
            chainlist.append(line[21])
            prev_chain=line[21]
pdbfh.close()
print(chainlist)

['B', 'C', 'D', 'E']


In [223]:
#!/usr/bin/env python3
#checking 3f9v
import os
import sys
#sys.argv[1]
pdbid = "../mcm_test/pdb_bank/equal_atoms/1-mer/3F9V_stndrd_A_withgaps" 
pdbf = pdbid+".pdb"
pdbfh = open(pdbf, 'r')
prev_res=None
count=1
for line in pdbfh:
    if line.startswith('ATOM'):
        if line[22:26] != prev_res:
            count=1
        if line[22:26] == prev_res:
            count=count+1
            if count > 4:
                print(line[22:26])
        prev_res=line[22:26]
#res71 in equalatoms
#res50 in cutandcleaned

  71
  71
  71
  71


In [245]:
#!/usr/bin/env python3
#checking 3f9v
import os
import sys
#sys.argv[1]
pdbid = "3F9V_stndrd-cut" 
pdbf = pdbid+".pdb"
pdbfh = open(pdbf, 'r')
prev_res=None
count=1
for line in pdbfh:
    if line.startswith('ATOM'):
        if line[22:26] != prev_res:
            count=1
        if line[22:26] == prev_res:
            count=count+1
            if count > 5:
                print(line[22:26])
        prev_res=line[22:26]
#res71 in equalatoms
#res50 in cutandcleaned

 321
 321
 321
 321
 321


In [226]:
#!/usr/bin/env python3
#checking 3f9v
import os
import sys
#sys.argv[1]
pdbid = "../mcm_test/pdb_bank/crude/3F9V" 
pdbf = pdbid+".pdb"
pdbfh = open(pdbf, 'r')
prev_res=None
count=1
for line in pdbfh:
    if line.startswith('ATOM'):
        if line[22:26] != prev_res:
            count=1
        if line[22:26] == prev_res:
            count=count+1
            if count > 5:
                print(line[22:26])
        prev_res=line[22:26]
#res327 in crude

In [772]:
#!/usr/bin/env python3
import os
import sys
import re
from Bio import AlignIO

#af = sys.argv[1]
af = "../mcm_test/pdb_bank/aligned/alignment_files/align_output.fasta"
align = AlignIO.read(af, "fasta")
args=["905:950","1735:1956","2608:2703","3510:3633","4189:4289","4891:5073"]

#translate numbers
i_start_list=list()
i_stop_list=list()
pdbid="3F9V_stndrd" #os.environ["id"]
pdb=pdbid + ".pdb"
sequence=""
res_tick=0
tick=0
psuedo_n=0
translated=list()
for record in align:
    if record.id == pdbid:
        #get i set 
        for i in args:#sys.argv[2:]:
            i_split=re.split(':', i)
            i_start=int(i_split[0]) 
            i_stop=int(i_split[1])
            i_start_list.append(i_start)
            i_stop_list.append(i_stop)

        #translate alignment column # to pdb res #
        for n in range(1, i_stop_list[-1]+1):
            subtractor=0
            #count residues 
            if record.seq[n-1] == '-':
                #adder=1
                pass
            else:
                res_tick=res_tick+1
            #put important residues in pdb
            if n == i_start_list[tick]:  
                for x in range(res_tick, (res_tick+(i_stop_list[tick]-(i_start_list[tick]))+1)):
                    if psuedo_n < n:
                        psuedo_n=n
                    if record.seq[psuedo_n-1] == '-':
                        if n != psuedo_n:
                            subtractor=subtractor+1
                        pass
                    else:
                        if subtractor>=x:
                            res=x
                            print("WARNING: the subtractor is larger than x. this is extremely odd and likely indicative of an issue")
                        else:
                            res=x-subtractor
                        translated.append(res)
                    psuedo_n=psuedo_n+1
                tick=tick+1
                if tick+1 > len(i_start_list):
                    break
#write new pdb          
if not translated:
    print(pdbid," does not contain any residues in the specified range. Omitting from rest of process")
    os.remove(pdb)
    sys.exit()
newpdb=pdbid + "-cut" + ".pdb"
f = open(newpdb, "w")
pdbfh = open(pdb, 'r')
prev_writ_t=translated[0]-1
prev_t=translated[0]
ter=False
terline='TER'+' '*77
for line in pdbfh:
    if line.startswith('ATOM'):
        for t in translated:
            if int(line[22:26]) == t:
                if prev_t != t:
                    if t != prev_t+1:
                        if not ter:    #prevents double TER
                            f.write(terline+'\n')
                            ter=True
                prev_t=t
                f.write(line)
                ter=False
            else:
                continue
    elif line.startswith('TER'):
        if not ter:
            f.write(terline+'\n')
            ter=True
    else:
        print("potentially strange line:",line)
        f.write('\n')
f.close()    
pdbfh.close()
#3 324 1100 272
#44 365 1955 321

In [774]:
#!/usr/bin/env python3
#checking 3f9v
import os
import sys
#sys.argv[1]
pdbid = "3F9V_stndrd-cut" 
pdbf = pdbid+".pdb"
pdbfh = open(pdbf, 'r')
prev_res=None
count=1
for line in pdbfh:
    if line.startswith('ATOM'):
        if line[22:26] != prev_res:
            count=1
        if line[22:26] == prev_res:
            count=count+1
            if count > 5:
                print(line[22:26])
        prev_res=line[22:26]
#res71 in equalatoms
#res50 in cutandcleaned

In [771]:
print(translated)

[1]


In [750]:
t5=translated
#t1=3933
#t2=3935
#t3=3932
#t4=1101
#t5=905

In [751]:
t4==t5

True

In [734]:
print(t2)
print(t3)

[273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559]
[273, 274, 275, 276

In [661]:
i_start_list

[905, 1735, 2608, 3510, 4189, 4891]

In [662]:
i_stop_list

[950, 1956, 2703, 3633, 4289, 5073]

In [615]:
#print()
res_tick=1
print(res_tick+1, (res_tick+1+(i_stop_list[0]-i_start_list[0]+1)+1))

2 49


In [616]:
i_stop_list[0]-i_start_list[0]

45

In [652]:
pdbid="3F9V_stndrd"
for record in align:
    if record.id == pdbid:
        n=594 #is an I
        print(record.seq[n-1])
        print(record.seq[n])

I
D


In [638]:
#xx-----xxx-
#want res's 3 to 5
#istart=n=8
#at istart=8, res_tick=3 bc its counting it
#istop=10


for x in range(3,3+(10-8)+1):
    print(x)
#gives 3,4,5

3
4
5


In [663]:
#-xx-----xxx-
#want res's 3 to 5
#istart=n=9
#at istart=9, res_tick=3 bc its counting it
#istop=11


for x in range(3,3+(11-9)+1):
    print(x)
#gives 3,4,5


3
4
5


In [647]:
#-xx-----xxx-
#want res's 3 to 5
#istart=n=8 which includes first gap
#at istart=8, res_tick=2 bc its not counting it
#istop=11

s=1
for x in range(2,2+(11-8)+1):
    if x==2:
        x=x-s
    print(x)


1
3
4
5


In [648]:
#-xx-----xxx-
#want res's 3 to 5
#istart=n=7 which includes first 2 gap
#at istart=7, res_tick=2 bc its not counting it
#istop=11

s=1
for x in range(2,2+(11-7)+1):
    if x==2:
        x=s-1
    print(x)


0
3
4
5
6


In [664]:
#-xx-----xxx-
#want res's 3 to 5
#istart=n=7 which includes first 2 gap
#at istart=7, res_tick=2 bc its not counting it
#istop=11

s=1
for x in range(2,2+(11-7)+1):
    print(x)
#gives 3,4,5


2
3
4
5
6


In [578]:
for x in range(1,3+1):
    print(x)

1
2
3
